In [1]:
# !pip install -U deepctr-torch
# !git clone https://github.com/jaeyoung-kang/career_recommendation.git

# import sys
# sys.path.append('./career_recommendation')

In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import confusion_matrix

from src.model import DeepFMTrainer
from src.utils import MajorCleaner
from src.dataset.augmentation import make_binary_target

/Users/m/.pyenv/versions/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
data_path = '/Users/m/localspace/career_recommendation/data/school_variable_major.csv'
epochs = 30

# Data Load

In [5]:
data = pd.read_csv(data_path).sort_values('id')

In [6]:
data['field'] = data['field'].str.split(',')
data = data.explode('field')
data = make_binary_target(data, 'field')
data['field'] = data['field'].str.strip()

In [7]:
data['school_major_name'] = data['school_major_name'].str.split(',')

# Data Split

In [8]:
index_list = list(range(len(data)))
random.shuffle(index_list)

data = data.iloc[index_list]

In [9]:
train_len = int(len(data) * 0.8)
train_data = data.iloc[:train_len]
test_data = data.iloc[train_len:]

# Trainer

In [10]:
target = ['target']
sparse_features = [
    "school_major_state", #"school_name",
    "school_major_level", "school_start",
    "school_end", "school_state", "field",
]
variable_length_sparse_feature = "school_major_name"


In [11]:
deepfm = DeepFMTrainer(
    target=target,
    sparse_features=sparse_features,
    variable_length_feature=variable_length_sparse_feature,
)

# Fit

In [12]:
deepfm.fit(
    train_data, 
    epochs=epochs,
)

Label Encoding ...

Build Model ...

Model Input ...
	exmple)
	 school_major_state :  6
	 school_major_level :  7
	 school_start :  33
	 school_end :  29
	 school_state :  7
	 field :  5
	 school_major_name :  [46  0  0  0]

cpu
Train on 105088 samples, validate on 26273 samples, 411 steps per epoch
Epoch 1/30
7s - loss:  0.6464 - accuracy:  0.6270 - val_accuracy:  0.6470
Epoch 2/30
7s - loss:  0.6180 - accuracy:  0.6525 - val_accuracy:  0.6667
Epoch 3/30
7s - loss:  0.6094 - accuracy:  0.6688 - val_accuracy:  0.6693
Epoch 4/30
7s - loss:  0.6009 - accuracy:  0.6730 - val_accuracy:  0.6729
Epoch 5/30
7s - loss:  0.5927 - accuracy:  0.6778 - val_accuracy:  0.6800
Epoch 6/30
7s - loss:  0.5893 - accuracy:  0.6802 - val_accuracy:  0.6814
Epoch 7/30
7s - loss:  0.5879 - accuracy:  0.6819 - val_accuracy:  0.6818
Epoch 8/30
7s - loss:  0.5871 - accuracy:  0.6821 - val_accuracy:  0.6815
Epoch 9/30
7s - loss:  0.5862 - accuracy:  0.6821 - val_accuracy:  0.6821
Epoch 10/30
7s - loss:  0.5849 - 

# Predict

In [13]:
predict = deepfm.predict(
    test_data,
)

## Evaluate

In [14]:
predict[predict < 0.5] = 0
predict[predict >= 0.5] = 1

tn, fp, fn, tp = confusion_matrix(
    test_data[target].values, predict,
).ravel()


acc = sum([tn, tp]) / sum([tn, fp, fn, tp])
print(acc)

0.6739441551718888


In [15]:
tn, fp, fn, tp 

(10118, 6327, 4381, 12015)